In [1]:
!rm -rf $PIP_BUILD/watson-machine-learning-client

In [2]:
#Install the WML client if required.
!pip install watson-machine-learning-client --upgrade

    100% |████████████████████████████████| 573kB 1.6MB/s eta 0:00:01
Requirement not upgraded as not directly required: lomond in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: urllib3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: ibm-cos-sdk in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: certifi in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-learning-client)
Requirement not upgraded as not directly required: tabulate in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from watson-machine-

In [3]:
wml_credentials = {
  "url": "https://ibm-watson-ml.mybluemix.net",
  "username": "52d46c59-49a8-4fec-a28d-5498e217a426",
  "password": "efee6497-827a-4f31-997f-d8cb7cff852a",
  "instance_id": "ae740039-e875-48e3-82a5-c646d9bee8a3"
}

In [4]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [5]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

In [6]:
# List information about your WML service instance.
print(wml_client.instance_details)

{'metadata': {'url': 'https://ibm-watson-ml.mybluemix.net/v3/wml_instances/ae740039-e875-48e3-82a5-c646d9bee8a3', 'modified_at': '2018-04-16T15:29:31.404Z', 'guid': 'ae740039-e875-48e3-82a5-c646d9bee8a3', 'created_at': '2018-04-16T15:29:31.404Z'}, 'entity': {'organization_guid': '391b59f1-c040-4fad-acb4-9fccc63826fc', 'plan': 'lite', 'region': 'us-south', 'owner': {'country_code': 'USA', 'ibm_id': '502K958BYM', 'user_id': '2f4badfc-a95d-4620-aa3f-1d28e7c650f8', 'email': 'haya3@nationwide.com', 'beta_user': False}, 'usage': {'gpu_count': {'limit': 8, 'current': 0}, 'computation_time': {'limit': 180000, 'current': 0}, 'capacity_units': {'limit': 180000000, 'current': 0}, 'expiration_date': '2018-05-01T00:00:00.000Z', 'deployment_count': {'limit': 5, 'current': 0}, 'prediction_count': {'limit': 5000, 'current': 0}, 'model_count': {'limit': 200, 'current': 0}}, 'status': 'Active', 'account': {'name': '', 'type': 'PAYG', 'id': '0027c35d0b236714fd648c4ceaced5b3'}, 'published_models': {'url':

In [7]:
# List information about the stored models.
wml_client.repository.list_models()

----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


In [8]:
# List the deployed models.
wml_client.deployments.list()

----  ----  ----  -----  -------  ---------
GUID  NAME  TYPE  STATE  CREATED  FRAMEWORK
----  ----  ----  -----  -------  ---------


In [11]:
!pip install wget

  Running setup.py bdist_wheel for wget ... done
  Stored in directory: /home/dsxuser/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [15]:
# Download sample PMML model, iris_chaid.xml, from Git.
import wget, os

sample_dir = 'pmml_sample_model'
if not os.path.isdir(sample_dir):
    os.mkdir(sample_dir)
    
filename=os.path.join(sample_dir, 'iris_chaid.xml')
if not os.path.isfile(filename):
    filename = wget.download('https://github.com/pmservice/wml-sample-models/raw/master/pmml/iris-species/model/iris_chaid.xml', out=sample_dir)

In [16]:
# Save the downloaded file to the WML repository.
props_pmml = {wml_client.repository.ModelMetaNames.NAME: 'CHAID PMML model for Iris data',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "IBM", 
              wml_client.repository.ModelMetaNames.AUTHOR_EMAIL: "ibm@ibm.com", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "pmml", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "4.2"}

model_details = wml_client.repository.store_model(filename, props_pmml)
wml_client.repository.list_models()

------------------------------------  ------------------------------  ------------------------  ---------
GUID                                  NAME                            CREATED                   FRAMEWORK
8ed30e7d-0a26-4b4b-8a8e-946e53c4f98d  CHAID PMML model for Iris data  2018-04-16T15:51:56.704Z  pmml-4.2
------------------------------------  ------------------------------  ------------------------  ---------


In [17]:
# Create the online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Iris species prediction')

wml_client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '4136252c-d354-43c4-bcb9-16396d850079' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='4136252c-d354-43c4-bcb9-16396d850079'
------------------------------------------------------------------------------------------------


------------------------------------  -----------------------  ------  --------------  ------------------------  ---------
GUID                                  NAME                     TYPE    STATE           CREATED                   FRAMEWORK
4136252c-d354-43c4-bcb9-16396d850079  Iris species prediction  online  DEPLOY_SUCCESS  2018-04-16T15:54:10.253Z  pmml-4.2
------------------------------------  -

In [18]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://ibm-watson-ml.mybluemix.net/v3/wml_instances/ae740039-e875-48e3-82a5-c646d9bee8a3/published_models/8ed30e7d-0a26-4b4b-8a8e-946e53c4f98d/deployments/4136252c-d354-43c4-bcb9-16396d850079/online


In [21]:
# Score data and predict species of flower.
scoring_data = {'fields': ['Sepal.Length', 'Sepal.Width', 'Petal.Length', 'Petal.Width'], 'values': [[5.1, 3.5, 1.4, 0.2]]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': ['PredictedValue', 'Confidence'], 'values': [['setosa', 1.0]]}
